加载软件包，设置参数

In [0]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as trans
import torchvision.datasets as dsets
import torch.nn as nn
import time
from torch.optim import lr_scheduler
%matplotlib inline
batch_size = 500
Num_Classes = 10
input_dim = 28*28
nepoch = 10
lr = 1e-1
device = torch.device('cuda')

In [0]:
trainset = dsets.MNIST(root='../data/mnist',train=True,transform=trans.ToTensor(),download=True)
testset = dsets.MNIST(root='../data/mnist',train=False,transform=trans.ToTensor(),download=True)
train_loader = DataLoader(dataset=trainset,batch_size=batch_size,shuffle=True,num_workers=6)
test_loader = DataLoader(dataset=testset,batch_size=batch_size,num_workers=6)


In [10]:
class LogisticReg(nn.Module):
  def __init__(self,in_dim,out_dim):
    super(LogisticReg,self).__init__()
    self.fc = nn.Linear(in_dim,out_dim)

  def forward(self,x):
    x = x.view(-1,784)
    o = self.fc(x)
    return o

model = LogisticReg(input_dim,Num_Classes)
model = model.to(device)
print(model)


LogisticReg(
  (fc): Linear(in_features=784, out_features=10, bias=True)
)


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
scheduler = lr_scheduler.MultiStepLR(optimizer,milestones=[6],gamma=0.2)


In [0]:
def eval(model,criterion,dataloader):
  loss,acc = 0,0
  for bx,by in dataloader:
    bx,by = bx.to(device),by.to(device)
    logit = model(bx)
    err = criterion(logit,by)
    loss += err.item()
    _,pred_y = logit.max(dim=1)
    acc += ((pred_y==by).float().sum())/by.size(0)
  loss /= len(dataloader)
  acc /= len(dataloader)
  return loss,100*acc



In [17]:
for epoch in range(nepoch):
  l = 0
  time_st = time.time()
  for bx,by in train_loader:
    bx,by = bx.to(device),by.to(device)
    logit = model(bx)
    loss = criterion(logit,by)
    loss.backward()
    optimizer.step()
    l += loss.item()
    optimizer.zero_grad()

  scheduler.step()
  l /= len(train_loader)
  print("model eval:")
  train_loss,train_acc = eval(model,criterion,train_loader)
  test_loss,test_acc = eval(model,criterion,test_loader)
  time_ed = time.time()
  print("epoch:%d,time:%.3f,train_loss=%.5f,train_acc=%.3f \n test_loss=%.5f,test_acc=%.3f"%(
      epoch+1,time_ed-time_st,train_loss,train_acc,test_loss,test_acc
  ))


model eval:
epoch:1,time:15.324,train_loss=0.34271,train_acc=91.702 
 test_loss=0.48140,test_acc=90.470
model eval:
epoch:2,time:15.277,train_loss=0.35927,train_acc=91.485 
 test_loss=0.48174,test_acc=90.430
model eval:
epoch:3,time:15.300,train_loss=0.41399,train_acc=90.267 
 test_loss=0.58085,test_acc=88.720



KeyboardInterrupt



In [0]:
train_loss,train_acc = eval(model,criterion,train_loader)
test_loss,train_acc = eval(model,criterion.test_loader)
print("epoch:%d,train_loss=%.5f,train_acc=%.3f,\n,test_loss=%.5f,test_acc=%.3f"%(
    epoch+1,train_loss,train_acc,test_loss,test_acc
))